# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
!pip install jupysql
!pip install duckdb-engine


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [5]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [6]:
%%sql
create or replace table competencia_02 as 
select 
    *
from read_csv_auto('C:\Users\Kalou\OneDrive\Escritorio\DMEyF\TareasHogar\Primera entrega\competencia_02_crudo.csv')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [7]:
%sql select * from competencia_02 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,0.0,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,0.0,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,0.0,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,0.0,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,0.0,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66


In [8]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_02
group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


In [9]:
%%sql
create or replace table targets AS
WITH aux AS  (
    SELECT numero_de_cliente, foto_mes,
    (foto_mes//100-2019)*12+foto_mes%100 as mes,
    IF(foto_mes = (select max(foto_mes) from competencia_02) , -1, lead(mes,1,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+1',
    IF(foto_mes = (select max(foto_mes) from competencia_02) OR foto_mes = ((select max(foto_mes) from competencia_02)-1), -1,lead(mes,2,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+2',
    FROM
       competencia_02
    ORDER BY numero_de_cliente, foto_mes)
SELECT numero_de_cliente, foto_mes,
CASE
    -- Condiciones de borde:
    WHEN ("mes+1"==-1) THEN 'CONTINUA'
    WHEN ("mes+2"==-1) THEN 'CONTINUA'
    -- Si mes siguiente no es consecutivo o no existe:
    WHEN ("mes"+1 != "mes+1") THEN 'BAJA+1'
    WHEN ("mes"+2 != "mes+2") THEN 'BAJA+2'
    -- si proximos 2 meses consecutivos activos:
    ELSE 'CONTINUA' END AS "clase_ternaria"
FROM aux
LEFT JOIN competencia_02 c USING (numero_de_cliente, foto_mes);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [10]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)


,Success


In [11]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes;


,Success


In [12]:
%%sql
select clase_ternaria, count(*)
from competencia_02
group by clase_ternaria

,clase_ternaria,count_star()
0,CONTINUA,4524980
1,BAJA+2,18987
2,BAJA+1,18843


In [13]:
%%sql
copy competencia_02 to 'C:\Users\Kalou\OneDrive\Escritorio\DMEyF\datasets\competencia_02.csv'(FORMAT CSV, HEADER)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
